In [13]:
# Q2 – Setup
import pandas as pd
import numpy as np
from math import ceil
from sklearn.neighbors import BallTree
from gurobipy import Model, GRB, quicksum

# ---- Parameters ----
DIST_MIN_MI = 0.06                  
EARTH_RADIUS_MI = 3959.0
RAD_EPS = DIST_MIN_MI / EARTH_RADIUS_MI
EQUIP_COST_PER_05 = 100.0           

In [14]:
# Load datasets and clean columns
df_pop = pd.read_csv('/Users/fengxiaopei/Desktop/population.csv')
df_inc = pd.read_csv('/Users/fengxiaopei/Desktop/avg_individual_income.csv')
df_emp = pd.read_csv('/Users/fengxiaopei/Desktop/employment_rate.csv')
df_fac = pd.read_csv('/Users/fengxiaopei/Desktop/child_care_regulated.csv')
df_pot = pd.read_csv('/Users/fengxiaopei/Desktop/potential_locations.csv')

# unify column names
df_inc = df_inc.rename(columns={'ZIP code':'zipcode','average income':'avg_income'})
df_emp = df_emp.rename(columns={'employment rate':'employment_rate'})
if 'zip_code' in df_fac.columns:
    df_fac = df_fac.rename(columns={'zip_code':'zipcode'})

# keep first 5 digits of zip
for df in [df_pop, df_inc, df_emp, df_fac, df_pot]:
    df['zipcode'] = df['zipcode'].astype(str).str[:5]

In [15]:
# Compute demand and existing capacity
data = df_pop[['zipcode','-5','5-9','10-14']].copy()
data = data.merge(df_inc[['zipcode','avg_income']], on='zipcode', how='left')
data = data.merge(df_emp[['zipcode','employment_rate']], on='zipcode', how='left')

data['avg_income'] = data['avg_income'].fillna(data['avg_income'].median())
data['employment_rate'] = data['employment_rate'].fillna(data['employment_rate'].median())

data['children_0_5']   = data['-5']
data['children_5_9']   = data['5-9']
data['children_10_12'] = (3/5) * data['10-14']
data['children_5_12']  = data['children_5_9'] + data['children_10_12']
data['children_0_12']  = data['children_0_5'] + data['children_5_12']

data['high_demand'] = ((data['avg_income'] <= 60000) | (data['employment_rate'] >= 0.60)).astype(int)
data['required_total_slots'] = data.apply(
    lambda r: ceil(0.5 * r['children_0_12']) if r['high_demand'] else ceil(0.33 * r['children_0_12']),
    axis=1
)
data['required_0_5_slots'] = np.ceil((2/3) * data['children_0_5']).astype(int)

# Existing facility summary
for c in ['total_capacity','infant_capacity','toddler_capacity','preschool_capacity']:
    if c not in df_fac.columns: df_fac[c] = 0
df_fac = df_fac.fillna(0)
df_fac['cap_0_5'] = df_fac['infant_capacity'] + df_fac['toddler_capacity']

existing_summary = df_fac.groupby('zipcode').agg(
    existing_total_slots=('total_capacity','sum'),
    existing_0_5_slots=('cap_0_5','sum')
).reset_index()

data = data.merge(existing_summary, on='zipcode', how='left').fillna(0)

In [16]:
# Build facility and potential site lists + spacing pairs
if 'facility_id' not in df_fac.columns:
    df_fac = df_fac.reset_index().rename(columns={'index':'facility_id'})

fac_df = df_fac[['facility_id','zipcode','total_capacity','cap_0_5','latitude','longitude']].copy()
fac_df = fac_df.rename(columns={'total_capacity':'nf','cap_0_5':'n5'})


fac_df['nf'] = pd.to_numeric(fac_df['nf'], errors='coerce').fillna(0)
fac_df = fac_df[fac_df['nf'] > 0].copy()          

df_pot = df_pot[['zipcode','latitude','longitude']].copy()
df_pot['site_id'] = np.arange(len(df_pot))

sites_in_zip = df_pot.groupby('zipcode')['site_id'].apply(list).to_dict()
facs_in_zip  = fac_df.groupby('zipcode')['facility_id'].apply(list).to_dict()

too_close_pairs, forbidden_sites = [], set()
for z, S in sites_in_zip.items():
    if not S: continue
    sub_sites = df_pot.set_index('site_id').loc[S, ['latitude','longitude']]
    sub_sites = sub_sites.dropna()
    if sub_sites.empty: continue
    site_ids = sub_sites.index.to_numpy(int)
    site_coords = np.radians(sub_sites[['latitude','longitude']].to_numpy())
    tree = BallTree(site_coords, metric='haversine')
    neigh = tree.query_radius(site_coords, r=RAD_EPS)
    for a, js in enumerate(neigh):
        for b in js:
            if b <= a: continue
            too_close_pairs.append((site_ids[a], site_ids[b]))

forbidden_sites = set()

fac_coord_df = fac_df.dropna(subset=['latitude','longitude'])[['facility_id','zipcode','latitude','longitude']].copy()
fac_coord_df['facility_id'] = fac_coord_df['facility_id'].astype(int)

for z, S in sites_in_zip.items():
    if not S:
        continue
    sub_sites = df_pot.set_index('site_id').loc[S, ['latitude','longitude']].dropna()
    if sub_sites.empty:
        continue

    # 本 zip 的既有设施（有经纬度）
    Fz = fac_coord_df[fac_coord_df['zipcode']==z]
    if Fz.empty:
        continue

    site_ids = sub_sites.index.to_numpy(int)
    site_coords = np.radians(sub_sites[['latitude','longitude']].to_numpy())

    fac_coords = np.radians(Fz[['latitude','longitude']].to_numpy())
    tree_fac = BallTree(fac_coords, metric='haversine')
    neigh = tree_fac.query_radius(site_coords, r=RAD_EPS)

    for a, js in enumerate(neigh):
        if len(js) > 0:
            forbidden_sites.add(int(site_ids[a]))


In [17]:
# Define model and variables
m = Model("Q2_realistic_expansion")


exp1, exp2, exp3, exp05 = {}, {}, {}, {}
for _, r in fac_df.iterrows():
    f, nf = int(r['facility_id']), float(r['nf'])
    exp1[f]  = m.addVar(lb=0, ub=0.10*nf, name=f"exp1_{f}")
    exp2[f]  = m.addVar(lb=0, ub=0.05*nf, name=f"exp2_{f}")
    exp3[f]  = m.addVar(lb=0, ub=0.05*nf, name=f"exp3_{f}")
    exp05[f] = m.addVar(lb=0, ub=0.20*nf, name=f"exp05_{f}")  # 0–5 扩容

# new
yS, yM, yL = {}, {}, {}
for _, r in df_pot.iterrows():
    i = int(r['site_id'])
    yS[i] = m.addVar(vtype=GRB.BINARY, name=f"yS_{i}")
    yM[i] = m.addVar(vtype=GRB.BINARY, name=f"yM_{i}")
    yL[i] = m.addVar(vtype=GRB.BINARY, name=f"yL_{i}")
m.update()

In [18]:
# Objective: expansion + build + 0–5 equip cost
exp_cost = quicksum(
    (20000/float(r['nf']) + 200)*exp1[int(f)] +
    (20000/float(r['nf']) + 400)*exp2[int(f)] +
    (20000/float(r['nf']) + 1000)*exp3[int(f)]
    for f, r in fac_df.set_index('facility_id').iterrows()
)

# new cost
build_cost = (
    quicksum(65000*yS[i] for i in yS) +
    quicksum(95000*yM[i] for i in yM) +
    quicksum(115000*yL[i] for i in yL)
)

# 0–5 cost
new05 = lambda i: 50*yS[i] + 100*yM[i] + 200*yL[i]
equip_cost = EQUIP_COST_PER_05 * (quicksum(exp05[f] for f in exp05) + quicksum(new05(i) for i in yS))

m.setObjective(exp_cost + build_cost + equip_cost, GRB.MINIMIZE)

In [19]:
# Constraints
# 1) one size per site
for i in yS:
    m.addConstr(yS[i] + yM[i] + yL[i] <= 1)

# 2) 20% cap & 0–5 expansion bound
for _, r in fac_df.iterrows():
    f, nf = int(r['facility_id']), float(r['nf'])
    m.addConstr(exp1[f]+exp2[f]+exp3[f] <= 0.20*nf)
    m.addConstr(exp05[f] <= exp1[f]+exp2[f]+exp3[f])

# 3) per-zip demand
zip_data = data.set_index('zipcode').T.to_dict()
for z in data['zipcode']:
    reqT  = zip_data[z]['required_total_slots']
    req05 = zip_data[z]['required_0_5_slots']
    existT = zip_data[z]['existing_total_slots']
    exist05 = zip_data[z]['existing_0_5_slots']

    Fz = fac_df.loc[fac_df['zipcode']==z, 'facility_id'].tolist()
    Sz = sites_in_zip.get(z, [])

    m.addConstr(existT + quicksum(exp1[f]+exp2[f]+exp3[f] for f in Fz) +
                quicksum(100*yS[i]+200*yM[i]+400*yL[i] for i in Sz) >= reqT)

    m.addConstr(exist05 + quicksum(exp05[f] for f in Fz) +
                quicksum(50*yS[i]+100*yM[i]+200*yL[i] for i in Sz) >= req05)

# 4) distance
for (i,j) in too_close_pairs:
    m.addConstr(yS[i]+yM[i]+yL[i] + yS[j]+yM[j]+yL[j] <= 1)
    
# forbid sites that are too close to existing facilities
for i in forbidden_sites:
    m.addConstr(yS[i] + yM[i] + yL[i] == 0, name=f"forbid_site_{i}")

In [20]:
# Solve & summarize
m.optimize()
if m.status == GRB.OPTIMAL:
    print(f"Optimal total cost: ${m.objVal:,.0f}")
else:
    print("Model not optimal. Status:", m.status)

# Quick breakdown
print("Expansion 0–5 slots:", sum(v.x for v in exp05.values()))
print("Total new facilities built:",
      sum(yS[i].x>0.5 or yM[i].x>0.5 or yL[i].x>0.5 for i in yS))

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 24.2.0 24C101)

CPU model: Intel(R) Core(TM) i7-1060NG7 CPU @ 1.20GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 335482 rows, 708612 columns and 2303162 nonzeros
Model fingerprint: 0x5bfafd5b
Variable types: 62412 continuous, 646200 integer (646200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+02, 1e+05]
  Bounds range     [2e-01, 2e+02]
  RHS range        [6e-01, 1e+04]
Presolve removed 264671 rows and 503472 columns (presolve time = 5s)...
Presolve removed 304192 rows and 614956 columns (presolve time = 72s)...
Presolve removed 304191 rows and 614955 columns
Presolve time: 72.02s
Presolved: 31291 rows, 93657 columns, 227876 nonzeros
Variable types: 9653 continuous, 84004 integer (83964 binary)
Found heuristic solution: objective 5.336609e+08
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal